In [3]:
import matplotlib.pyplot as plt
import mne
import numpy as np
from scipy.stats import ttest_rel
# from braindecode.datasets import MOABBDataset

from moabb.datasets import Wang2016

In [4]:
# dataset = MOABBDataset(dataset_name="Nakanishi2015", subject_ids=[1])

<!-- fname = C:\Users\Sunsun\mne_data\MNE-nakanishi-data\mnakanishi\12JFPM_SSVEP\raw\master\data\s1.mat -->

In [6]:
dataset = Wang2016()

for i in range(1, 36):
    subject = [i]
    dataset.subject_list = subject
    sessions = dataset.get_data(subjects=subject)
    session_name = "session_0"
    run_name = "run_0"
    raw_list = [sessions[id][session_name][run_name] for id in sessions]

    raw = mne.concatenate_raws(raw_list)
    raw.info['line_freq'] = 60.

    # Set montage
    montage = mne.channels.make_standard_montage('standard_1005')
    raw.set_montage(montage, verbose=False)

    # Set common average reference
    raw.set_eeg_reference('average', projection=False, verbose=False)

    # Apply bandpass filter
    raw.filter(l_freq=0.1, h_freq=None, fir_design='firwin', verbose=False)

    # Construct epochs
    event_id = {
        '8hz': 1,
        '15.8hz': 40
    }
    # events, _ = mne.events_from_annotations(raw, verbose=False)
    events = mne.find_events(raw)
    tmin, tmax = -1., 5.  # in s
    baseline = None
    epochs = mne.Epochs(
        raw, events=events,
        event_id=[event_id['8hz'], event_id['15.8hz']], tmin=tmin,
        tmax=tmax, baseline=baseline, verbose=False)
    
    tmin = 1.
    tmax = 5.
    fmin = 1.
    fmax = 90.
    sfreq = epochs.info['sfreq']

    spectrum = epochs.compute_psd(
        'welch',
        n_fft=int(sfreq * (tmax - tmin)),
        n_overlap=0, n_per_seg=None,
        tmin=tmin, tmax=tmax,
        fmin=fmin, fmax=fmax,
        window='boxcar',
        verbose=False)
    psds, freqs = spectrum.get_data(return_freqs=True)
    snrs = snr_spectrum(psds, noise_n_neighbor_freqs=3,
                    noise_skip_neighbor_freqs=1)
    fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(8, 5))
    freq_range = range(np.where(np.floor(freqs) == 1.)[0][0],
                    np.where(np.ceil(freqs) == fmax - 1)[0][0])

    psds_plot = 10 * np.log10(psds)
    psds_mean = psds_plot.mean(axis=(0, 1))[freq_range]
    psds_std = psds_plot.std(axis=(0, 1))[freq_range]
    axes[0].plot(freqs[freq_range], psds_mean, color='b')
    axes[0].fill_between(
        freqs[freq_range], psds_mean - psds_std, psds_mean + psds_std,
        color='b', alpha=.2)
    axes[0].set(title="PSD spectrum", ylabel='Power Spectral Density [dB]')

    # SNR spectrum
    snr_mean = snrs.mean(axis=(0, 1))[freq_range]
    snr_std = snrs.std(axis=(0, 1))[freq_range]

    axes[1].plot(freqs[freq_range], snr_mean, color='r')
    axes[1].fill_between(
        freqs[freq_range], snr_mean - snr_std, snr_mean + snr_std,
        color='r', alpha=.2)
    axes[1].set(
        title="SNR spectrum", xlabel='Frequency [Hz]',
        ylabel='SNR', ylim=[-2, 30], xlim=[fmin, fmax])
    fig

Trial data de-meaned and concatenated with a buffer to create continuous data


240 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


Trial data de-meaned and concatenated with a buffer to create continuous data


240 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


Trial data de-meaned and concatenated with a buffer to create continuous data


240 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


Trial data de-meaned and concatenated with a buffer to create continuous data


In [ ]:
snrs = snr_spectrum(psds, noise_n_neighbor_freqs=3,
                    noise_skip_neighbor_freqs=1)